In [1]:
import os
import sys
import numpy as np
from tqdm import tqdm
import time
import pickle
import easydict

In [2]:
import logging

In [3]:
logger = logging.getLogger(__name__)
logging.basicConfig(format="%(asctime)s [%(levelname)s] %(message)s", 
                    level=logging.INFO,
                    handlers=[
                        logging.FileHandler(os.path.join("./subtext_test_result.log")),
                        logging.StreamHandler()
                    ])

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

## Call model (BertSum/SubtextDivider)

### (1) BertSum

In [4]:
# path 추가
sys.path.append('/home/sks/korea_univ/21_1/TA/team_project/youtube_summarizer/src/bertsum')
sys.path.append('/home/sks/korea_univ/21_1/TA/team_project/youtube_summarizer/src/subtext')

In [5]:
from models.data_loader import TextLoader, load_dataset
from src.backbone import ExtTransformerEncoder, ExtSummarizer, WindowEmbedder

2021-05-07 01:07:47,033 [INFO] PyTorch version 1.1.0 available.


In [6]:
args = easydict.EasyDict({
    "visible_gpus" : -1,
    "temp_dir" : './tmp/',
    "test_from": None,
    "max_pos" : 512,
    "large" : False,
    "finetune_bert": True,
    "encoder": "bert",
    "share_emb": False,
    "dec_layers": 6,
    "dec_dropout": 0.2,
    "dec_hidden_size": 768,
    "dec_heads": 8,
    "dec_ff_size": 2048,
    "enc_hidden_size": 512,
    "enc_ff_size": 512,
    "enc_dropout": 0.2,
    "enc_layers": 6,
    
    "ext_dropout": 0.2,
    "ext_layers": 2,
    "ext_hidden_size": 768,
    "ext_heads": 8,
    "ext_ff_size": 2048,
    
    "accum_count": 1,
    "save_checkpoint_steps": 5,
    
    "generator_shard_size": 32,
    "alpha": 0.6,
    "beam_size": 5,
    "min_length": 15,
    "max_length": 150,
    "max_tgt_len": 140,  
    "block_trigram": True,
    
    "model_path": "./tmp_model/",
    "result_path": "./tmp_result/src",
    "recall_eval": False,
    "report_every": 1,
})

In [10]:
# Settings
device = "cpu" if args.visible_gpus == -1 else "cuda"
loader = TextLoader(args, device)

# model setting
ckpt_path = '/home/sks/korea_univ/21_1/TA/team_project/youtube_summarizer/src/bertsum/checkpoint/model_step_24000.pt'
checkpoint = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
bert_model = ExtSummarizer(args, device, checkpoint)
bert_model.eval()

using cached model
using cached model
using cached model
using cached model


2021-05-07 01:07:58,074 [INFO] loading configuration file ./tmp/kobert_from_pretrained/config.json
2021-05-07 01:07:58,077 [INFO] Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 2,
  "vocab_size": 8002
}

2021-05-07 01:07:58,079 [INFO] loading weights file ./tmp/kobert_from_pretrained/pytorch_model.bin


using cached model


2021-05-07 01:07:59,349 [INFO] All model checkpoint weights were used when initializing BertModel.

2021-05-07 01:07:59,350 [INFO] All the weights of BertModel were initialized from the model checkpoint at ./tmp/kobert_from_pretrained.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use BertModel for predictions without further training.


ExtSummarizer(
  (bert): Bert(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(8004, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): LayerNorm(torch.Size([768]),

In [11]:
embedder = WindowEmbedder(model=bert_model, text_loader=loader)

### (2) SubtextDivider

In [12]:
from model.subtext_classifier import SubtextClassifier

subtext_model = SubtextClassifier(window_size=3).cuda()

model_path = '/home/sks/korea_univ/21_1/TA/team_project/youtube_summarizer/src/subtext/ckpt/subtext_model_w3.pt'
subtext_model.load_state_dict(torch.load(model_path))

subtext_model.eval()

SubtextClassifier(
  (block1): Sequential(
    (0): Conv1d(768, 128, kernel_size=(4,), stride=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block2): Sequential(
    (0): Linear(in_features=384, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=1, bias=True)
  )
)

### (3) Score Calculator

In [13]:
def get_divscore(src_doc=[], embedder=None, divider=None):
    embedding = embedder.get_embeddings(src_doc).transpose(1, 0).unsqueeze(0)
    score = divider(embedding).item()
    return score

# Dataset for evaluation

In [14]:
import json
def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

In [15]:
news_df = load_jsonl('/home/sks/korea_univ/21_1/TA/team_project/youtube_summarizer/dataset/article_dataset/train.jsonl')

Loaded 260697 records from /home/sks/korea_univ/21_1/TA/team_project/youtube_summarizer/dataset/article_dataset/train.jsonl


In [16]:
# 전처리
# (1) 글자 개수가 너무 작은 경우 없애기 (30글자 이상)
# (2) 문장이 적은 경우 해당 기사 없애기 (10문장 이상)
news_clean = []
for news in news_df:
    news_article = news['article_original']
    if len(news_article) >= 10:
        article_clean = [sent for sent in news_article if len(sent) >= 30]
        news_clean.append(article_clean)

In [17]:
import random

def make_mixed_doc(news_dataset=None, max_num=1000):
    mixed_doc_set = []
    for i in range(max_num):
        lh_count = min(random.randint(7, 10), len(news_dataset[i]))
        rh_count = min(random.randint(7, 10), len(news_dataset[i+1]))

        lh_news = news_dataset[i][:lh_count]
        rh_news = news_dataset[i+1][:rh_count]
        
        gt = lh_count - 1

        src_doc = '\n'.join((lh_news + rh_news))
        mixed_doc_set.append((src_doc, gt))
        
    return mixed_doc_set

In [18]:
random.seed(2020011135)
mixed_doc_list = make_mixed_doc(news_dataset=news_clean, max_num=1000)

## Evaluation

In [36]:
# Settings
loader = TextLoader(args, device)
window_size = 3

using cached model
using cached model


In [18]:
err_cnt = 0
acc_cnt = 0
ws = 3

div_result = []
for i, a_set in enumerate(mixed_doc_list):
    
    if (i+1) % 20 == 0:
        logger.info(f"working on {i+1}th doc: Accuracy so far is {acc_cnt/(acc_cnt+err_cnt)*100:.2f}%")
        
    src_doc = a_set[0].split('\n')
    gt = a_set[1]
    
    cands = [src_doc[i:i+ws*2] for i, _ in enumerate(src_doc) if i <= len(src_doc) - ws*2]
    
    # 가끔 한문장이 너무길어서 잘리는 경우가 있음..
    try:
        div_scores = [get_divscore(src_doc=cand, embedder=embedder, divider=subtext_model) for cand in cands]
        div_point = div_scores.index(max(div_scores)) + ws - 1

        if div_point == gt:
            acc_cnt += 1
        else:
            err_cnt += 1
            
    except RuntimeError as e:
        print(f"Error occurred at {i}th article")
    
#     sents = [sent for sent in src_doc.split('\n') if sent]
#     lh_sent, rh_sent = [], []
#     for i, sent in enumerate(sents):
#         if i <= div_point:
#             lh_sent.append(sent)
#         else:
#             rh_sent.append(sent)
            
#     result_sents = lh_sent + ["----------------[DIV]---------------"] + rh_sent
#     div_result.append((result_sents, div_scores, div_point, gt))

2021-05-02 19:57:07,751 [INFO] working on 20th doc: Accuracy so far is 73.68%


Error occurred at 30th article
Error occurred at 32th article
Error occurred at 33th article


2021-05-02 19:57:52,749 [INFO] working on 40th doc: Accuracy so far is 77.78%
2021-05-02 19:58:43,279 [INFO] working on 60th doc: Accuracy so far is 80.36%


Error occurred at 66th article
Error occurred at 67th article


2021-05-02 19:59:27,099 [INFO] working on 80th doc: Accuracy so far is 75.68%
2021-05-02 20:00:19,010 [INFO] working on 100th doc: Accuracy so far is 79.79%
2021-05-02 20:01:05,288 [INFO] working on 120th doc: Accuracy so far is 79.82%
2021-05-02 20:01:53,274 [INFO] working on 140th doc: Accuracy so far is 78.36%


...

2021-05-02 20:31:04,383 [INFO] working on 880th doc: Accuracy so far is 80.33%
2021-05-02 20:31:47,949 [INFO] working on 900th doc: Accuracy so far is 79.98%
2021-05-02 20:32:33,937 [INFO] working on 920th doc: Accuracy so far is 79.64%
2021-05-02 20:33:25,227 [INFO] working on 940th doc: Accuracy so far is 79.76%
2021-05-02 20:34:13,123 [INFO] working on 960th doc: Accuracy so far is 79.33%


Error occurred at 967th article
Error occurred at 968th article


2021-05-02 20:34:59,979 [INFO] working on 980th doc: Accuracy so far is 79.51%


Error occurred at 983th article
Error occurred at 984th article


2021-05-02 20:35:49,014 [INFO] working on 1000th doc: Accuracy so far is 79.27%


In [7]:
print(f"{acc_cnt / (acc_cnt + err_cnt)*100:.2f}%")

NameError: name 'acc_cnt' is not defined

### (1) 기사

In [93]:
script_list = mixed_doc_list[35][0].split('\n')

In [94]:
script_list

['[충청일보 이정규기자] 폐원을 신청한 청주 은성유치원이 충북도교육감을 상대로 낸 행정소송을 취하하려했지만 도교육청이 이를 거부한 것으로 전해졌다.',
 "6일 충북도교육청에 따르면 은성유치원은 지난 2017년 7월 도교육감을 상대로 '징계의결 요구 처분 취소' 청구소송을 낸 후 지난해 12월 27일 소취하 신청서를 재판부에 제출했다.",
 '그러나 지난해 12월 4일 도교육청이 소취하에 동의하지 않는다는 의견을 내면서 재판이 재개됐다.',
 '도교육청은 2017년 사립유치원 종합감사를 통해 은성유치원 회계 비리를 적발하고, 원장 정직을 유치원측에 요구했다.',
 '은성유치원은 이에 반발해 도교육청에 대해 행정소송을 제기했다.',
 '도교육청이 소 취하를 거부한 이유는 소송을 진행해 사립유치원 감사 업무 기준이 되는 판례를 만들겠다는 것이다.',
 '교육청 관계자는 "판례가 있게 되면 행정 추진 기준으로 삼을 수 있다"며 "법원의 최종 판단을 받기 위해 소취하에 부동의했다"고 전했다.',
 "은성유치원은 오는 28일까지 폐원하겠다며 청주시교육지원청에 '학교 폐쇄 인가신청서'를 제출했다.",
 '이날은 재개된 소송의 다음 재판이 열리는 변론기일이다.',
 '재판 중 폐원이 완료되면 소송을 유지할 이익이 사라지기 때문에 법원이 선고 때 본안 판단없이 각하 결정할 공산이 크다.',
 "배우 인교진과 최대철이 드라마 '동백꽃 필 무렵'에서 특급 카메오로 활약 했다.",
 "인교진과 최대철은 지난 30일 방송된 KBS2 수목극 '동백꽃 필 무렵'(극본 임상춘, 연출 차영훈)에서 강하늘의 두 형으로 등장했다.",
 '아버지 제삿날을 맞아 덕순(고두심 분)의 식당에 찾아온 그림이었다.',
 '그 시간 용식(강하늘 분)은 동백(공효진 분)을 만나려다 정숙(이정은 분)의 "동백이를 만나면 어머니는? 어중간하게 착하려면 그만 둬"라는 말에 뭔가 결심한 듯 돌아왔고, 동백과 만남을 허락해달라고 엄마 덕순의 식당 문을 연 순간 형들과 마주쳤다.',
 '꽃을 든 용식을 본 둘째 형

In [95]:
score_list = []
for i, sent in enumerate(tqdm(script_list)):
    if i + (ws*2) <= len(script_list):
        w_input = script_list[i:i+(ws*2)]
        
        # embedding
        emb = embedder.get_embeddings(w_input).transpose(1, 0).cuda()
        score = subtext_model(emb.unsqueeze(0))
        score_list.append(score.item())

100%|██████████| 18/18 [00:02<00:00,  7.36it/s]


In [96]:
score_list

[-4.86625862121582,
 -5.065357685089111,
 -5.856250762939453,
 -4.983221530914307,
 -4.679341793060303,
 -4.1713433265686035,
 -4.891319751739502,
 8.072851181030273,
 -3.0970635414123535,
 -3.884166955947876,
 -7.703949928283691,
 -5.633543491363525,
 -6.8576555252075195]

In [97]:
for i, sent in enumerate(script_list):
    if (i >= ws) & (i <= len(script_list) - ws):
        j = (i-ws)
        print(f"{score_list[j]:.2f}", '\n', sent)
    else:
        print(sent)

[충청일보 이정규기자] 폐원을 신청한 청주 은성유치원이 충북도교육감을 상대로 낸 행정소송을 취하하려했지만 도교육청이 이를 거부한 것으로 전해졌다.
6일 충북도교육청에 따르면 은성유치원은 지난 2017년 7월 도교육감을 상대로 '징계의결 요구 처분 취소' 청구소송을 낸 후 지난해 12월 27일 소취하 신청서를 재판부에 제출했다.
그러나 지난해 12월 4일 도교육청이 소취하에 동의하지 않는다는 의견을 내면서 재판이 재개됐다.
-4.87 
 도교육청은 2017년 사립유치원 종합감사를 통해 은성유치원 회계 비리를 적발하고, 원장 정직을 유치원측에 요구했다.
-5.07 
 은성유치원은 이에 반발해 도교육청에 대해 행정소송을 제기했다.
-5.86 
 도교육청이 소 취하를 거부한 이유는 소송을 진행해 사립유치원 감사 업무 기준이 되는 판례를 만들겠다는 것이다.
-4.98 
 교육청 관계자는 "판례가 있게 되면 행정 추진 기준으로 삼을 수 있다"며 "법원의 최종 판단을 받기 위해 소취하에 부동의했다"고 전했다.
-4.68 
 은성유치원은 오는 28일까지 폐원하겠다며 청주시교육지원청에 '학교 폐쇄 인가신청서'를 제출했다.
-4.17 
 이날은 재개된 소송의 다음 재판이 열리는 변론기일이다.
-4.89 
 재판 중 폐원이 완료되면 소송을 유지할 이익이 사라지기 때문에 법원이 선고 때 본안 판단없이 각하 결정할 공산이 크다.
8.07 
 배우 인교진과 최대철이 드라마 '동백꽃 필 무렵'에서 특급 카메오로 활약 했다.
-3.10 
 인교진과 최대철은 지난 30일 방송된 KBS2 수목극 '동백꽃 필 무렵'(극본 임상춘, 연출 차영훈)에서 강하늘의 두 형으로 등장했다.
-3.88 
 아버지 제삿날을 맞아 덕순(고두심 분)의 식당에 찾아온 그림이었다.
-7.70 
 그 시간 용식(강하늘 분)은 동백(공효진 분)을 만나려다 정숙(이정은 분)의 "동백이를 만나면 어머니는? 어중간하게 착하려면 그만 둬"라는 말에 뭔가 결심한 듯 돌아왔고, 동백과 만남을 허락해달라고 엄마 덕순의 식당 문을 연

In [38]:
ws=3
tmp_src = mixed_doc_list[100][0].split('\n')
tmp_cands = [tmp_src[i:i+ws*2] for i, _ in enumerate(tmp_src) if i <= len(tmp_src) - ws*2]

In [40]:
idx = [get_divscore(src_doc=cand, embedder=embedder, divider=subtext_model) for cand in tmp_cands]

### (2) 유튜브

In [19]:
from utils.preprocess import doc_preprocess
import json

In [52]:
youtube_script_pth = '/home/sks/korea_univ/21_1/TA/team_project/youtube_summarizer/dataset/youtube_dataset/label/KBS뉴스_sNMcImF7Ubo_27m_38s.txt'
with open(youtube_script_pth, 'rb') as rr:
    youtube_df = json.load(rr)

In [53]:
script = youtube_df['text']

In [54]:
script_fin = doc_preprocess(script)

In [55]:
script_list = [sent for sent in script_fin.split('\n') if len(sent.strip()) >= 20]#[:50]
#script_list = [sent for sent in script_fin.split('\n')][:50]

In [56]:
ws = 3

In [57]:
subtext_model.eval()

score_list = []
for i, sent in enumerate(tqdm(script_list)):
    if i + (ws*2) <= len(script_list):
        w_input = script_list[i:i+(ws*2)]
        
        # embedding
        emb = embedder.get_embeddings(w_input).transpose(1, 0).cuda()
        score = subtext_model(emb.unsqueeze(0))
        score_list.append(score.item())

100%|██████████| 184/184 [00:25<00:00,  7.11it/s]


In [58]:
score_list

[-0.3943195343017578,
 -6.566971778869629,
 -5.9564361572265625,
 -4.123239994049072,
 -5.181784629821777,
 -5.251582145690918,
 -6.510199546813965,
 -6.372408390045166,
 0.538477897644043,
 -5.2188310623168945,
 -5.503244400024414,
 -5.466092586517334,
 -4.190450668334961,
 8.113215446472168,
 -4.470927715301514,
 -1.239018201828003,
 -4.1706671714782715,
 -1.8685868978500366,
 -1.1108489036560059,
 -3.2292702198028564,
 -4.812061309814453,
 -2.418874979019165,
 3.236466407775879,
 -5.411825180053711,
 -6.229593753814697,
 -1.3415755033493042,
 -6.690732955932617,
 -5.865991592407227,
 -5.601990699768066,
 -6.884272575378418,
 -6.069212913513184,
 -5.230625152587891,
 -5.326739311218262,
 -6.234976768493652,
 1.0425372123718262,
 -6.210102081298828,
 -4.568972110748291,
 -5.4841814041137695,
 -6.131746292114258,
 -6.544376373291016,
 -6.337329387664795,
 -6.241468906402588,
 -6.131739616394043,
 -5.225009441375732,
 -4.9916605949401855,
 1.2107954025268555,
 -2.4454524517059326,
 5.63

In [59]:
for i, sent in enumerate(script_list):
    if (i >= ws) & (i <= len(script_list) - ws):
        j = (i-ws)
        if score_list[j] > 0:
            print(f"========({score_list[j]:.2f})========", '\n', sent)
        else:
            print(f"{score_list[j]:.2f}", '\n', sent)
    else:
        print(sent)

여러분 안녕하십니까 코로나19 통합뉴스룸 5시 뉴스입니다
미국이 한국을 비롯한 세계 각국의 코로나19 백신을 제공하는 방안과 관련해 자국 우선 사용 원칙을 분명히 했습니다
바이든 대통령 역시 현재로서는 해외 공급이 어렵다면서도 앞으로 그렇게 되길 바란다고 말했습니다
-0.39 
 미 국무부가 코로나19 백신의 해외 공유와 관련해 국내 공급이 우선이라고 밝혔습니다
-6.57 
 레드 프라이스 미 국무부 대변인은 코로나19 백신 확보를 위한 한미 백신 수확 후 협정 체결 가능성에 대한 질문에 포괄적으로 답하겠다며 이같이 말했습니다
-5.96 
 그러면서 바이든 정부는 현재 최대한 짧은 시간 안에 가장 빠른 속도로 미국 내 접종자 수를 늘리는 데 주력하고 있다고 설명했습니다
-4.12 
 56만여 명이 숨지고 3천만 명 넘는 확진자가 나온 미국 상황을 진정시키는 게 세계적 확산 방지 협력보다 중요하다는 점을 재확인한 겁니다
-5.18 
 이 같은 정책 방향을 반영하듯 현지시간 21일 미국 내 코로나19 백신 접종은 2억 회를 넘어섰습니다
-5.25 
 바이든 대통령은 역사적인 백신 접종 노력의 새 장을 열었다고 자평하면서도 해외 공유 가능성에 대해선 회의적 시각을 드러냈습니다
-6.51 
 바이든 대통령은 다만 각국의 상황에 도움을 주기 위해 안전성을 전제로 사용하지 않는 백신 가운데 일부를 어떻게 할지 살펴보는 중이라고 덧붙였습니다
-6.37 
 세계 각국이 빗발치는 요청에도 불구하고 미국이 집단 면역 전에 백신을 해외에 대규모로 공급하는 상황은 사실상 기대하기 어려워 보입니다
========(0.54)======== 
 코로나19 신규 확진자는 오늘도 700명대로 지난 1월 첫째 주 이후 가장 높은 수치를 기록했습니다
-5.22 
 백신 수급 비판에 대해 정부는 이미 충분한 백신을 공급받기로 했다며 추가 확보를 위해 신규 백신 개발 동향도 주시하고 있다고 밝혔습니다
-5.50 
 오늘 0시 기준 코로나19 신규 확진자는 735명 어제보다 4명 늘어난 것으로 지난